# Footy tipping!

Creating features for the finals.

Not worrying about finals location for the time being. Hopefully the ladder position & finals week can control for that.

In [34]:
# Render our plots inline
%matplotlib inline
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make the graphs a bit prettier
pd.set_option('display.mpl_style', 'default') 
plt.rcParams['figure.figsize'] = (15, 5)
# Limit the size of the dataframe html output in the ipython notebook.
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 20)
# Loading the data
teams = pd.read_csv('data/teams.csv')
seasons = pd.read_csv('data/seasons.csv')
finals = pd.read_csv('data/finals.csv')

In [35]:
df_finals = finals[["mid", 'season', 'round', 'tid1_loc', 'tid1', 'tid2', 'win_tid', 'h_tid', 'h_score', 'a_tid', 'a_score', 'margin' ]]

In [36]:
df_finals.loc[:, 'prob'] = 0
df_finals.loc[(df_finals['win_tid'] == df_finals['tid1']), 'prob'] = 1    

Creating the year's data set

In [37]:
# Simplifying the data set
df_train = seasons[["mid", 'season', 'round', 'tid1_loc', 'tid1', 'tid2', 'win_tid', 'h_tid', 'h_score', 'a_tid', 'a_score', 'margin' ]]

In [38]:
sLength = len(df_train['mid'])
df_train.ix[:, 'tid1_score'] = 0 #pd.Series(np.zeros(sLength), index=df_train.index)
df_train.ix[:, 'tid2_score'] = 0 #pd.Series(np.zeros(sLength), index=df_train.index)

In [39]:
home_is_tid1 = (df_train['h_tid'] == df_train['tid1'])
away_is_tid1 = (df_train['a_tid'] == df_train['tid1'])
df_train.ix[home_is_tid1,'tid1_score'] = df_train['h_score']
df_train.ix[away_is_tid1,'tid1_score'] = df_train['a_score']
df_train.ix[away_is_tid1,'tid2_score'] = df_train['h_score']
df_train.ix[home_is_tid1,'tid2_score'] = df_train['a_score']

In [40]:
# Calculating last year's percentage
# scores for
tid1_scores = df_train.groupby(['season', 'tid1']).sum()['tid1_score']
tid2_scores = df_train.groupby(['season', 'tid2']).sum()['tid2_score']
scores_for = pd.concat([tid1_scores, tid2_scores], axis=1)
scores_for['scores_for'] = 0
scores_for.scores_for = scores_for.tid1_score.fillna(0) + scores_for.tid2_score.fillna(0)
#scores against
tid1_game_tid2_opponent_score = df_train.groupby(['season', 'tid1']).sum()['tid2_score']
tid2_game_tid1_opponent_score = df_train.groupby(['season', 'tid2']).sum()['tid1_score']
scores_against = pd.concat([tid1_game_tid2_opponent_score, tid2_game_tid1_opponent_score], axis=1)
scores_against['scores_against'] = 0
scores_against.scores_against = scores_against.tid1_score.fillna(0) + scores_against.tid2_score.fillna(0)
current_year = pd.concat([scores_for, scores_against], axis=1)
current_year = current_year[['scores_for', 'scores_against']]
current_year['percentage'] = current_year['scores_for'] / current_year['scores_against']
current_year = current_year[['percentage']]

In [41]:
df_train.loc[:, 'tid1_season_percentage'] = 0
df_train.loc[:, 'tid2_season_percentage'] = 0
teams_list = pd.unique(teams['tid'].ravel())
teams_list_pre_2011 = np.delete(teams_list, [7,8])
teams_list_pre_2012 = np.delete(teams_list, [8])
seasons_list = pd.unique(df_train.season.ravel())
base_year = 2005

for year in range(base_year, 2015+1):
    for team in teams_list_pre_2011:
        df_train.loc[(df_train['season']==year) & (df_train['tid1']==team), 'tid1_season_percentage'] = current_year.loc[year, team][0]            
        df_train.loc[(df_train['season']==year) & (df_train['tid2']==team), 'tid2_season_percentage'] = current_year.loc[year, team][0]            

In [42]:
# Bloody Gold Coast
for year in range(2011, 2015+1):
    df_train.loc[(df_train['season']==year) & (df_train['tid1']==108), 'tid1_season_percentage'] = current_year.loc[year, 108][0]            
    df_train.loc[(df_train['season']==year) & (df_train['tid2']==108), 'tid2_season_percentage'] = current_year.loc[year, 108][0] 

In [43]:
# Bloody GWS
# Give them their first year's percentage as the lagged percentage
for year in range(2012, 2015+1):
    df_train.loc[(df_train['season']==year) & (df_train['tid1']==109), 'tid1_season_percentage'] = current_year.loc[year, 109][0]             
    df_train.loc[(df_train['season']==year) & (df_train['tid2']==109), 'tid2_season_percentage'] = current_year.loc[year, 109][0]

### Calculating the win ratio for the year to date

win ratio = $\frac{\# wins + 0.5\cdot\#draws}{4\cdot \#games}$

Base year

In [44]:
# Collecting premiership points. Ignoring draws.
df_train.loc[:,'tid1_points'] = 0
df_train.loc[(df_train['tid1']==df_train['win_tid']), 'tid1_points'] = 4
df_train.loc[:,'tid2_points'] = 0
df_train.loc[(df_train['tid2']==df_train['win_tid']), 'tid2_points'] = 4
df_train.loc[(df_train['margin']== 0), 'tid1_points'] = 2
df_train.loc[(df_train['margin']== 0), 'tid2_points'] = 2
tid1_points = df_train.groupby(['season', 'tid1']).sum()['tid1_points']
tid2_points = df_train.groupby(['season', 'tid2']).sum()['tid2_points']
tid_points = pd.concat([tid1_points, tid2_points], axis=1)
tid_points['tid_points'] = 0
tid_points.tid_points = tid_points.tid1_points.fillna(0) + tid_points.tid2_points.fillna(0)
# dividing by total amount of points available. 22 games a season at 4 points each
tid_points['total_points'] = (4 * 22.0)
tid_points['win_ratio'] = tid_points.tid_points / tid_points.total_points
tid_points = tid_points[['win_ratio']]

In [45]:
df_train.loc[:, 'tid1_season_win_ratio'] = 0
df_train.loc[:, 'tid2_season_win_ratio'] = 0
teams_list = pd.unique(teams['tid'].ravel())
teams_list_pre_2011 = np.delete(teams_list, [7,8])
teams_list_pre_2012 = np.delete(teams_list, [8])
seasons_list = pd.unique(df_train.season.ravel())
base_year = 2000

for year in range(base_year, 2015+1):
    for team in teams_list_pre_2011:
        df_train.loc[(df_train['season']==year) & (df_train['tid1']==team), 'tid1_season_win_ratio'] = tid_points.loc[year, team][0]            
        df_train.loc[(df_train['season']==year) & (df_train['tid2']==team), 'tid2_season_win_ratio'] = tid_points.loc[year, team][0]            

In [46]:
# Bloody Gold Coast
# Give them their first year's percentage as the lagged percentage
for year in range(2011, 2015+1):
    df_train.loc[(df_train['season']==year) & (df_train['tid1']==108), 'tid1_season_win_ratio'] = tid_points.loc[year, 108][0]            
    df_train.loc[(df_train['season']==year) & (df_train['tid2']==108), 'tid2_season_win_ratio'] = tid_points.loc[year, 108][0] 

In [47]:
# Bloody GWS
# Give them their first year's percentage as the lagged percentage
for year in range(2012, 2015+1):
    df_train.loc[(df_train['season']==year) & (df_train['tid1']==109), 'tid1_season_win_ratio'] = tid_points.loc[year, 109][0]             
    df_train.loc[(df_train['season']==year) & (df_train['tid2']==109), 'tid2_season_win_ratio'] = tid_points.loc[year, 109][0]

Making a ladder

In [48]:
df_ladder = pd.DataFrame(columns=['season', 'tid', 'tid_season_win_ratio', 'tid_season_percentage', 'ladder'])

for year in range(base_year, 2015+1):
    # Running over two rounds incase of byes
    df_test = df_train
    df_test = df_test[(df_test['season']==year) & (df_test['round']=='R20')]
    df_test_1 = df_test[['season', 'tid1', 'tid1_season_win_ratio', 'tid1_season_percentage']]
    df_test_1.columns = ['season', 'tid', 'tid_season_win_ratio', 'tid_season_percentage']
    df_test_2 = df_test[['season', 'tid2', 'tid2_season_win_ratio', 'tid2_season_percentage']]
    df_test_2.columns = ['season', 'tid', 'tid_season_win_ratio', 'tid_season_percentage']
    df_test = pd.concat([df_test_1, df_test_2])
    df_test_A = df_test.sort(['tid_season_win_ratio', 'tid_season_percentage'])
    df_test = df_train
    df_test = df_test[(df_test['season']==year) & (df_test['round']=='R21')]
    df_test_1 = df_test[['season', 'tid1', 'tid1_season_win_ratio', 'tid1_season_percentage']]
    df_test_1.columns = ['season', 'tid', 'tid_season_win_ratio', 'tid_season_percentage']
    df_test_2 = df_test[['season', 'tid2', 'tid2_season_win_ratio', 'tid2_season_percentage']]
    df_test_2.columns = ['season', 'tid', 'tid_season_win_ratio', 'tid_season_percentage']
    df_test = pd.concat([df_test_1, df_test_2])
    df_test_B = df_test.sort(['tid_season_win_ratio', 'tid_season_percentage'])
    # dropping dupicates and creating the ladder
    df_ladder_0 = pd.concat([df_test_A, df_test_B]).sort(['tid_season_win_ratio', 'tid_season_percentage']).drop_duplicates()
    df_ladder_0.loc[:,'ladder'] = 0
    ladder = np.array(list(reversed(range(1, 1 + len(df_ladder_0['ladder'])))))
    df_ladder_0.loc[:, 'ladder'] = ladder
    df_ladder = pd.concat([df_ladder, df_ladder_0])

In [49]:
df_ladder1 = df_ladder[['season', 'tid', 'ladder']]
df_ladder1.columns = ['season', 'tid1', 'tid1_ladder']
df_ladder2 = df_ladder[['season', 'tid', 'ladder']]
df_ladder2.columns = ['season', 'tid2', 'tid2_ladder']

In [50]:
df_train = pd.merge(df_train, df_ladder1, on=['season', 'tid1'])
df_train = pd.merge(df_train, df_ladder2, on=['season', 'tid2'])

In [51]:
df_train = df_train[['mid', 'season', 'tid1', 'tid2', 'tid1_season_percentage', 'tid2_season_percentage', 'tid1_season_win_ratio', 'tid2_season_win_ratio', 'tid1_ladder', 'tid2_ladder']]

In [52]:
df_train_2015 = df_train[df_train['season']==2015]

In [53]:
df_test = df_train
df_train = pd.merge(df_finals, df_train, on=['season', 'tid1', 'tid2'], how = 'inner')

Extracting the round from the data

In [54]:
df_train = df_train.drop(['h_tid', 'h_score', 'mid_y', 'a_tid', 'a_score', 'margin'], axis=1)

In [55]:
df_train.loc[:, 'week'] = 0
df_train.loc[(df_train['round'] == 'QF'), 'week'] = 1
df_train.loc[(df_train['round'] == 'EF'), 'week'] = 1
df_train.loc[(df_train['round'] == 'SF'), 'week'] = 2
df_train.loc[(df_train['round'] == 'PF'), 'week'] = 3
df_train.loc[(df_train['round'] == 'GF'), 'week'] = 4
df_train.loc[(df_train['round'] == 'GFR'), 'week'] = 5

In [56]:
df_train = df_train.drop_duplicates()
df_train['mid_x'] = df_train['mid_x'].str.slice(8)

In [57]:
df_train = pd.get_dummies(df_train, columns=["tid1", "tid2", "tid1_loc"])

Developing the training, cross validation and test data

In [58]:
# Constructing a training, validation and hold out sets
df_cv = df_train[df_train["season"] == 2014].reset_index(drop=True)
df_train = df_train[(df_train["season"] >= 2000) & 
                   df_train["season"] <= 2013].reset_index(drop=True)

In [59]:
df_train.to_pickle('afl_train_finals.pkl')
df_cv.to_pickle('afl_cval_finals.pkl')

Setting up input submission data

In [60]:
df_train_2015['mid'] = df_train_2015['mid'].str.slice(9)
df_train_2015 = df_train_2015.drop_duplicates()

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [62]:
df_train_2015.to_pickle('afl_test_finals.pkl')